In [1]:
library(dplyr)
library(streamMetabolizer)
library(splitstackshape)
library(ggplot2)
library(lubridate)
library(reshape2)
library(neonUtilities)
library(progress)
library(patchwork)
library(doBy)
library(box)



Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


USGS Active Research Package:
https://owi.usgs.gov/R/packages.html#research

This package was developed for research purposes. We used it for our
own applications and welcome flexible, resilient users who can help us
test and improve the package. Please give us feedback at
https://github.com/USGS-R/streamMetabolizer/issues/new.



ERROR: Error in library(splitstackshape): there is no package called 'splitstackshape'


# Setting up the workspace
In the cell below indicate the site that will be analyzed here according to its NEON 4-letter code.

If you downloaded the repo into your home director, which is found by going to '~/', then you don't have to change the box path. FYI: Box is a new package for R that allows you to define a set of functions without creating a new package.

In [8]:
site_id<-"MART"
setwd(paste("~/gdrive/SUNA_data/SUNA_NEON/",site_id,sep=""))

options(box.path = '~/neon_suna/box/')




[1] 5

# Importing SUNA calibration data.

In [3]:

setwd("/Users/jhosen/gdrive/fdom/SUNA_NEON/MART")
sunas<-list.files(pattern="*.csv")

#suna_cal<-readRDS("/Users/jhosen/gdrive/fdom/SUNA Calibration pdfs/suna_cal_comb.rds")



In [4]:


compiled_suna<-data.frame()
for(i in 1:length(sunas)){
suna<-read.csv(paste(sunas[i]),skip=0,header=FALSE)

suna_cal<-read.csv(paste("/Users/jhosen/gdrive/fdom/SUNA_NEON/MART/MART_CAL/",unique(suna[,ncol(suna)]),sep=""))
suna_cal_t<-as.data.frame(t(suna_cal))
names(suna_cal_t)<-suna_cal_t[1,]

#suna_d0<-cSplit(suna,3, ",")
names(suna)<-c("suna_id","date","time","nitrate","nitrate_mgl","a254","a350","bromide_trace","spec_average_dark","dark_signal_average","inter_time",suna_cal$wavelength_nm,"sensor_temp","spec_temp","lamp_temp","lamp_time","rel_hum","main_volt","lamp_volt","int_volt","main_current","fit_aux_1","fit_aux_2","fit_base_1","fit_base_2","fit_RMSE","CTD_Time","CTD_Salinity","CTD_Temp","CTD_Pressure","checksum","cal")


suna<-subset(suna,a254!=0)
doy<-gsub("2020","",suna$date,fixed=TRUE)
day<-as.POSIXct(paste(as.Date(as.numeric(doy),origin="2020-01-01"),"00:00:00"),tz="UTC")
#suna$day<-as.POSIXct('2019-05-10 00:00:00',tz="UTC")
suna$dtp<-day + (3600*as.numeric(suna$time))
attr(suna$dtp,"tzone") <- "Etc/GMT+8"

offse<-c(t(suna_cal_t[3,]))
suna2<-(suna[,grep("^[0-9]{3}",names(suna))]/suna$inter_time)-suna$dark_signal_average
suna_d0_norm<-sweep(suna2,2,FUN="/",offse)
names(suna_d0_norm)<-paste("n_",names(suna_cal_t),sep="")
suna_d0_int<-data.frame()
for(j in 1:nrow(suna_d0_norm)){
	flip<-as.data.frame(t(suna_d0_norm[j,]))
	names(flip)<-c("abs")
	flip$wl_nm<-gsub("n_","",row.names(flip),fixed=TRUE)
	flip_int<-as.data.frame(t(approx(flip$wl_nm,flip$abs,xout=seq(189,394,1),rule=2)$y))
	names(flip_int)<-paste("interp_",seq(189,394,1),sep="")
	suna_d0_int<-bind_rows(suna_d0_int,flip_int)
}
suna_d<-bind_cols(suna,suna_d0_int)
#suna_d$date<-as.character(suna_d[,c("date")])
#suna_d$time<-as.character(suna_d[,c("time")])
suna_d2<-suna_d %>% mutate_if(is.numeric,as.character)
compiled_suna<-bind_rows(compiled_suna,suna_d2)
}

compiled_suna2<-subset(compiled_suna,!is.na(bromide_trace) & a254>0 & a350>0)

compiled_suna2$interp_254<-as.numeric(compiled_suna2$interp_254)
compiled_suna2$a254<-as.numeric(compiled_suna2$a254)
compiled_suna2$sensor_temp<-as.numeric(compiled_suna2$sensor_temp)
summary(lm(a254~interp_254+I(interp_254^2)+I(interp_254^3)+sensor_temp,compiled_suna2))

compiled_suna2$a350<-as.numeric(compiled_suna2$a350)
compiled_suna2$interp_350<-as.numeric(compiled_suna2$interp_350)
compiled_suna2$sensor_temp<-as.numeric(compiled_suna2$sensor_temp)
compiled_suna2$date<-as.numeric(compiled_suna2$date)
compiled_suna2$inter_time<-as.numeric(compiled_suna2$inter_time)

summary(lm(a350~log10(interp_350)+I(log10(interp_350)^2)+I(log10(interp_350)^3),compiled_suna2))
summary(lm(a254~log10(interp_254)+I(log10(interp_254)^2)+I(log10(interp_254)^3),compiled_suna2))

a350_mod<-lm(a350~log10(interp_350)+I(log10(interp_350)^2)+I(log10(interp_350)^3),compiled_suna2)
a254_mod<-lm(a254~log10(interp_254)+I(log10(interp_254)^2)+I(log10(interp_254)^3),compiled_suna2)

cor_cols<-grep("^interp_",names(compiled_suna2))

compiled_suna2[,c(cor_cols)]<-compiled_suna2[,c(cor_cols)] %>% mutate_if(is.character,as.numeric)

for(i in 1:length(cor_cols)){
	prepdata<-data.frame(interp_350=compiled_suna2[,cor_cols[i]],interp_254=compiled_suna2[,cor_cols[i]])
	a350_pred<-predict(a350_mod,newdata=prepdata)
	a254_pred<-predict(a254_mod,newdata=prepdata)
	compiled_suna2[,c(paste(names(compiled_suna2)[cor_cols[i]],"_c350",sep=""))]<-a350_pred
	compiled_suna2[,c(paste(names(compiled_suna2)[cor_cols[i]],"_c254",sep=""))]<-a254_pred
}

Warning message in sweep(suna2, 2, FUN = "/", offse):
"STATS is longer than the extent of 'dim(x)[MARGIN]'"


ERROR: Error in names(suna_d0_norm) <- paste("n_", names(suna_cal_t), sep = ""): 'names' attribute [256] must be the same length as the vector [0]
